# Introduction
* Purpose: Create a local ChromaDB vector store with existing EU legal acts from S3 bucket
* Python Environment: regpy10ch

# Libraries

In [23]:
import pandas as pd
import numpy as np
import os
import json
import boto3
import chromadb
import logging
import traceback
from tqdm import tqdm
from datetime import datetime
from bs4 import BeautifulSoup
from multiprocessing import Pool
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
import hashlib
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader, UnstructuredXMLLoader
from langchain_huggingface import HuggingFaceEmbeddings

def extract_mt_code(mt):
    if pd.isna(mt):
        return None
    else:
        return mt[0:5]
    
def extract_mt_term(mt):
    if pd.isna(mt):
        return None
    else:
        return mt[5:]

def decompose_celex(celex):
    sector = celex[0]
    year = celex[1:5]
    return sector, year

def make_id(doc_path):
    return hashlib.md5(doc_path[32:].encode("utf-8")).hexdigest()[:12]

In [3]:
AWS_ACCESS_KEY_ID = os.getenv("S3_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("S3_SECRET_ACCESS_KEY")
bucket_name = 'regguru'
# Create an S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

la_object = 'eu/LEG_EN_HTML_20250721_04_08'
mtd_object = 'eu/LEG_MTD_20250709_22_36'

In [4]:
la_mtd_file = os.getenv("EU_LEGAL_ACT_METADATA_FILE")
la_mtd = pd.read_csv(la_mtd_file)
la_mtd["MT-code"] = la_mtd["MT"].apply(extract_mt_code)
la_mtd["MT-term"] = la_mtd["MT"].apply(extract_mt_term)
la_mtd['doc-path'] = "eu/LEG_EN_HTML_20250721_04_08/" + la_mtd['work-id'] + "/" + la_mtd['format'] + "/" + la_mtd['doc']
la_mtd['mtd-path'] = "eu/LEG_MTD_20250709_22_36/" + la_mtd['work-id'] + "/tree_non_inferred.rdf"

In [5]:
ch_client = chromadb.PersistentClient(path="./chroma_data")

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Slightly larger chunks
    chunk_overlap=200
)


# Test embed documents

In [ ]:
# ch_client.delete_collection(name="legal-test")

In [16]:
test_collection = ch_client.get_or_create_collection(name="legal-test") # default embedder: all-MiniLM-L6-v2
documents = la_mtd.drop_duplicates('doc-path').iloc[0:5]

In [17]:
for idx, row in tqdm(documents.iterrows(), desc="Processing documents", total=documents.shape[0]):
    # Load document from S3
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=row["doc-path"])
        html_content = obj['Body'].read().decode('utf-8')  # HTML as string
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator="\n")  # plain text
    except Exception as e:
        logging.error("%d | Error loading %s:\n%s", idx, row["doc-path"], traceback.format_exc())
        print(f"{idx} | Error loading {row['doc-path']}")
        continue
    
    # Extract metadata
    try:
        meta = {}
        if soup.title:
            meta['title'] = soup.title.string
        for m in soup.find_all("meta"):
            if m.get("name") and m.get("content"):
                meta[m["name"].lower()] = m["content"]
        meta_download = la_mtd[(la_mtd["work-id"] == row["work-id"]) & (la_mtd["doc"] == row["doc"])]
        terms = meta_download["TERMS (PT-NPT)"].dropna().astype(str).unique()
        meta['eurovoc-terms'] = ';'.join(terms)
        mts = meta_download["MT"].dropna().astype(str).unique()
        meta['eurovoc-mt'] = ';'.join(mts)

        sector, year = decompose_celex(row['celex']) if row['celex'] else (None, None)
        meta['celex'] = row['celex'] if row['celex'] else None
        meta['celex-sector'] = sector # a character
        meta['celex-year'] = int(year) # a 4-digit number
    except Exception as e:
        logging.error("%d | Error extracting metadata %s:\n%s", idx, row["doc-path"], traceback.format_exc())
        print(f"{idx} | Error extracting metadata {row['doc-path']}")


    # Split, embed, and store
    try:
        chunks = text_splitter.split_text(text)
        texts = [chunk for chunk in chunks]
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
        test_collection.add(
            documents=texts,
            metadatas=[meta for _ in range(len(texts))],
            embedding=embeddings.embed_documents(texts),
            ids=[f"{make_id(row['doc-path'])}_{i}" for i in range(len(texts))]
        )
    except Exception as e:
        logging.error("%d | Error embedding %s:\n%s", idx, row["doc-path"], traceback.format_exc())
        print(f"{idx} | Error embedding {row['doc-path']}")
        continue

Processing documents:  20%|██        | 1/5 [00:06<00:24,  6.17s/it]

0 | Error embedding eu/LEG_EN_HTML_20250721_04_08/1a1e8486-a474-11e9-9d01-01aa75ed71a1/xhtml/L_2019187EN.01004101.doc.html


Processing documents:  40%|████      | 2/5 [00:10<00:15,  5.18s/it]

6 | Error embedding eu/LEG_EN_HTML_20250721_04_08/5fa72f58-9564-4ebe-a5a5-853e206ae2ed/html/32000R0212en.html


Processing documents:  60%|██████    | 3/5 [00:15<00:09,  4.86s/it]

18 | Error embedding eu/LEG_EN_HTML_20250721_04_08/65a9ddff-3e78-403b-ab34-98d7973be2b3/xhtml/L_2010117EN.01006001.doc.html


Processing documents:  80%|████████  | 4/5 [00:19<00:04,  4.60s/it]

26 | Error embedding eu/LEG_EN_HTML_20250721_04_08/39bcdc85-2e3b-4e36-a488-5197ee502afd/html/31999Y0917_02_en.html


Processing documents: 100%|██████████| 5/5 [00:23<00:00,  4.78s/it]

31 | Error embedding eu/LEG_EN_HTML_20250721_04_08/a4d26b15-882d-11e9-9369-01aa75ed71a1/xhtml/L_2019148EN.01000101.doc.html


# Test chroma client

In [108]:
# test = la_mtd.drop_duplicates('doc-path').iloc[0:5]
test_collection = ch_client.get_or_create_collection(name="legal-test") # default embedder: all-MiniLM-L6-v2

In [109]:
print("Number of embeddings:", test_collection.count())
results = test_collection.peek()
print(results)

Number of embeddings: 26
{'ids': ['9000786f2ac2_0', '9000786f2ac2_1', '9000786f2ac2_2', '9000786f2ac2_3', '9000786f2ac2_4', '9000786f2ac2_5', '9000786f2ac2_6', '9000786f2ac2_7', '9000786f2ac2_8', '9000786f2ac2_9'], 'embeddings': array([[-0.03268745,  0.0328047 ,  0.02150211, ...,  0.00085366,
         0.0815249 ,  0.0492176 ],
       [-0.0428882 ,  0.06924605,  0.02441872, ...,  0.05517734,
         0.0789023 ,  0.00538935],
       [ 0.01047072,  0.00346978,  0.01473061, ..., -0.00961938,
         0.04713432,  0.01296261],
       ...,
       [ 0.00838298,  0.03890036, -0.04555006, ...,  0.08372673,
        -0.02075115, -0.00260066],
       [ 0.00876509,  0.00809463,  0.00598892, ...,  0.02392675,
         0.04353177, -0.00210776],
       [ 0.02035374, -0.00467432,  0.01529113, ..., -0.04337729,
         0.07403102,  0.04721521]], shape=(10, 384)), 'documents': ['L_2019187EN.01004101.xml\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n12.7.2019\xa0\xa0\xa0\n\n\n\n\n\n\nEN\n\n\n\n\n\n\nOffi

In [110]:
all_docs = test_collection.get()
for doc in all_docs['metadatas']:
    print(doc)

{'title': 'L_2019187EN.01004101.xml', 'celex-year': 2019, 'celex-sector': '3', 'celex': '32019D1194', 'eurovoc-mt': '5641 fisheries;2841 health;5631 agricultural activity;6811 chemistry;5216 deterioration of the environment;5206 environmental policy', 'eurovoc-terms': 'fish;endocrine disease;fish disease;chemical product;endocrine disruptor;environmental protection'}
{'celex-year': 2019, 'celex-sector': '3', 'celex': '32019D1194', 'title': 'L_2019187EN.01004101.xml', 'eurovoc-terms': 'fish;endocrine disease;fish disease;chemical product;endocrine disruptor;environmental protection', 'eurovoc-mt': '5641 fisheries;2841 health;5631 agricultural activity;6811 chemistry;5216 deterioration of the environment;5206 environmental policy'}
{'title': 'L_2019187EN.01004101.xml', 'celex': '32019D1194', 'eurovoc-mt': '5641 fisheries;2841 health;5631 agricultural activity;6811 chemistry;5216 deterioration of the environment;5206 environmental policy', 'celex-year': 2019, 'celex-sector': '3', 'eurovoc

In [111]:
# Semantic search
results = test_collection.query(
    query_texts=[
        "Which chemical was classified as a substance of very high concern for its endocrine-disrupting effects under the EU Commission Implementing Decision 2019/1194?",
        "On what date did EU Regulation 402/2010 become effective?"],
    n_results=5
)

results["documents"]

[['HAS ADOPTED THIS DECISION:\n\n\n\n\n\n\n\n\nArticle 1\n\n\n\n\n1.\xa0\xa0\xa04-tert-butylphenol (PTBP) (EC No 202-679-0, CAS No 98-54-4) is identified as a substance of very high concern pursuant to Article 57(f) of Regulation (EC) No 1907/2006 due to its endocrine disrupting properties with probable serious effects to the environment which give rise to an equivalent level of concern to those of other substances listed in Article 57(a) to (e) of that Regulation.\n\n\n\n\n\n\n2.\xa0\xa0\xa0The substance referred to in paragraph 1 shall be included in the candidate list referred to in Article 59(1) of Regulation (EC) No 1907/2006 with the following indication under ‘Reason for inclusion’: ‘Endocrine disrupting properties (Article 57(f) - environment)’.\n\n\n\n\n\n\n\n\nArticle 2\n\n\nThis Decision is addressed to the European Chemicals Agency.\n\n\n\n\n\n\n\n\n\n\nDone at Brussels, 5\xa0July 2019.\n\n\n\n\n\n\nFor the Commission\n\n\n\n\nElżbieta BIEŃKOWSKA\n            \n\n\n\n\nMemb

Metadata filtering

In [112]:
# Before limiting to a specific year
results = test_collection.query(
    query_texts=["civil servants"],
    n_results=5
)

results["documents"]

[['EUR-Lex - 31999Y0917(02) - EN\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAvis juridique important\n\n\n\n\n\n\n\n\n\n\n|\n\n\n\n\n\n\n\n\n31999Y0917(02)\n\n\n\n\nInformation from the Commission - Appointment of a new member of the Committee of Experts on the Transit of Electricity between Grids set up under Commission Decision 92/167/EEC (Text with EEA relevance)  \n\n\n\n\n\n\nOfficial Journal C 263 , 17/09/1999 P. 0003 - 0003\n \n\n\n\n\n\n\n\n\n\n\n\n\nAppointment of a new member of the Committee of Experts on the Transit of Electricity between Grids set up under Commission Decision 92/167/EEC\n(1999/C 263/03)\n(Text with EEA relevance)\nBy its decision of 13 September 1999 the Commission appointed to the position of new member of the Committee of Experts on the Transit of Electricity between Grids:\nas Spanish representative of the high-voltage grids,\nMr Victoriano Casajús, Director General of Transport, Red Electrica, to replace the outgoing Mr Novales.',
  'EUR-Lex - 32000R0212 - EN\n\n\n\

In [113]:
# After limiting to a specific year
results = test_collection.query(
    query_texts=["civil servants"],
    where={"celex-year": 2000},
    n_results=5
)

results["documents"]

[['EUR-Lex - 32000R0212 - EN\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAvis juridique important\n\n\n\n\n\n\n\n\n\n\n|\n\n\n\n\n\n\n\n\n32000R0212\n\n\n\n\nCouncil Regulation (EC, ECSC, Euratom) No 212/2000 of 24 January 2000 adjusting the daily subsistence allowance rates for officials on mission in Austria, Finland and Sweden laid down in Article 13 of Annex VII to the Staff Regulations of Officials of the European Communities  \n\n\n\n\n\n\nOfficial Journal L 024 , 29/01/2000 P. 0001 - 0002',
  'COUNCIL REGULATION (EC, ECSC, EURATOM) No 212/2000\nof 24 January 2000\nadjusting the daily subsistence allowance rates for officials on mission in Austria, Finland and Sweden laid down in Article 13 of Annex VII to the Staff Regulations of Officials of the European Communities\nTHE COUNCIL OF THE EUROPEAN UNION,\nHaving regard to the Treaty establising the European Community, and in particular Article 283 thereof,\nHaving regard to the Staff Regulations of Officials and the Conditions of Employment of O

In [87]:
# results = collection.query(
#     query_texts=["Tropical"],
#     where={"$and": [
#         {"color": "yellow"},
#         {"weight": {"$gt": 150}}
#     ]},
#     n_results=5
# )

# print(results["documents"][0])

In [114]:
# Full text search
results = test_collection.get(
    where_document={"$contains": "Union Customs Code"}
)

results["documents"]

['L_2019148EN.01000101.xml\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n6.6.2019\xa0\xa0\xa0\n\n\n\n\n\n\nEN\n\n\n\n\n\n\nOfficial Journal of the European Union\n\n\n\n\n\n\nL 148/1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            COMMISSION IMPLEMENTING REGULATION (EU) 2019/921\n\n\nof 3\xa0June 2019\n         \n\n\nconcerning the classification of certain goods in the Combined Nomenclature\n\n\n\n\n\n\nTHE EUROPEAN COMMISSION,\n\n\n\n\nHaving regard to the Treaty on the Functioning of the European Union,\n\n\n\n\n\n\nHaving regard to Regulation (EU) No 952/2013 of the European Parliament and of the Council of 9\xa0October 2013 laying down the Union Customs Code\xa0\n(\n1\n)\n, and in particular Article 57(4) and Article 58(2) thereof,\n\n\n\n\nWhereas:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n(1)\n\n\n\n\n\n\nIn order to ensure uniform application of the Combined Nomenclature annexed to Council Regulation (EEC) No 2658/87\xa0\n(\n2\n)\n, it is necessary to adopt measures concerning the classif

# Embed all documents

In [13]:
log_filename = datetime.now().strftime("errors_%Y%m%d_%H%M%S.log")
logging.basicConfig(
    filename=log_filename,          # file to write logs
    level=logging.ERROR,            # log only errors and above
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [ ]:
# ch_client.delete_collection(name="legal")
collection = ch_client.get_or_create_collection(name="legal")
documents = la_mtd.drop_duplicates('doc-path').reset_index(drop=True)
print(documents.shape[0], "documents to process")

46665 documents to process


## Loop
stop at 3067

In [16]:
for idx, row in tqdm(documents.iterrows(), desc="Processing documents", total=documents.shape[0]):
    # Load document from S3
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=row["doc-path"])
        html_content = obj['Body'].read().decode('utf-8')  # HTML as string
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator="\n")  # plain text
    except Exception as e:
        logging.error("%d | Error loading %s:\n%s", idx, row["doc-path"], traceback.format_exc())
        print(f"{idx} | Error loading {row['doc-path']} | {e}")
        continue
    
    # Extract metadata
    try:
        meta = {}
        if soup.title:
            meta['title'] = soup.title.string
        for m in soup.find_all("meta"):
            if m.get("name") and m.get("content"):
                meta[m["name"].lower()] = m["content"]
        meta_download = la_mtd[(la_mtd["work-id"] == row["work-id"]) & (la_mtd["doc"] == row["doc"])]
        terms = meta_download["TERMS (PT-NPT)"].dropna().astype(str).unique()
        meta['eurovoc-terms'] = ';'.join(terms)
        mts = meta_download["MT"].dropna().astype(str).unique()
        meta['eurovoc-mt'] = ';'.join(mts)

        sector, year = decompose_celex(row['celex']) if row['celex'] else (None, None)
        meta['celex'] = row['celex'] if row['celex'] else None
        meta['celex-sector'] = sector # a character
        meta['celex-year'] = int(year) # a 4-digit number
    except Exception as e:
        logging.error("%d | Error extracting metadata %s:\n%s", idx, row["doc-path"], traceback.format_exc())
        print(f"{idx} | Error extracting metadata {row['doc-path']} | {e}")


    # Split, embed, and store
    try:
        chunks = text_splitter.split_text(text)
        texts = [chunk for chunk in chunks]
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
        collection.add(
            documents=texts,
            metadatas=[meta for _ in range(len(texts))],
            embeddings=embeddings.embed_documents(texts),
            ids=[f"{make_id(row['doc-path'])}_{i}" for i in range(len(texts))]
        )
    except Exception as e:
        logging.error("%d | Error embedding %s:\n%s", idx, row["doc-path"], traceback.format_exc())
        print(f"{idx} | Error embedding {row['doc-path']} | {e}")
        continue

Processing documents:   0%|          | 165/46665 [29:10<90:08:02,  6.98s/it] ERROR:root:165 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000101.notes.0002.xml.jpg:
Traceback (most recent call last):
  File "/var/folders/zj/t8mz7g111911kcsdk6nwh8g00000gn/T/ipykernel_1963/64700856.py", line 5, in <module>
    html_content = obj['Body'].read().decode('utf-8')  # HTML as string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

ERROR:root:166 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000501.notes.0011.xml.jpg:
Traceback (most recent call last):
  File "/var/folders/zj/t8mz7g111911kcsdk6nwh8g00000gn/T/ipykernel_1963/64700856.py", line 5, in <module>
    html_content = obj['Body'].read().decode('utf-8')  # HTML as string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

Processing docu

165 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000101.notes.0002.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
166 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000501.notes.0011.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
167 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000501.notes.0001.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
168 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01001301.notes.0013.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
169 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01001301.notes.0003.xml.jpg | 'utf-8' codec can't d

ERROR:root:170 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01002401.notes.0003.xml.jpg:
Traceback (most recent call last):
  File "/var/folders/zj/t8mz7g111911kcsdk6nwh8g00000gn/T/ipykernel_1963/64700856.py", line 5, in <module>
    html_content = obj['Body'].read().decode('utf-8')  # HTML as string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

Processing documents:   0%|          | 171/46665 [29:10<20:54:46,  1.62s/it]ERROR:root:171 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000901.notes.0004.xml.jpg:
Traceback (most recent call last):
  File "/var/folders/zj/t8mz7g111911kcsdk6nwh8g00000gn/T/ipykernel_1963/64700856.py", line 5, in <module>
    html_content = obj['Body'].read().decode('utf-8')  # HTML as string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

ERROR:root:172 |

170 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01002401.notes.0003.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
171 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000901.notes.0004.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
172 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000501.notes.0027.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
173 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01002101.notes.0013.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


ERROR:root:174 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01002101.notes.0003.xml.jpg:
Traceback (most recent call last):
  File "/var/folders/zj/t8mz7g111911kcsdk6nwh8g00000gn/T/ipykernel_1963/64700856.py", line 5, in <module>
    html_content = obj['Body'].read().decode('utf-8')  # HTML as string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

Processing documents:   0%|          | 175/46665 [29:11<9:26:02,  1.37it/s] ERROR:root:175 | Error loading eu/LEG_EN_HTML_20250721_04_08/e9cc666d-ab4d-4716-a3e8-df42fc0e0747/xhtml/FOR-C_2007070EN.01000501.notes.0008.xml.jpg:
Traceback (most recent call last):
  File "/var/folders/zj/t8mz7g111911kcsdk6nwh8g00000gn/T/ipykernel_1963/64700856.py", line 5, in <module>
    html_content = obj['Body'].read().decode('utf-8')  # HTML as string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

ERROR:root:176 |

...

3055 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01002001.notes.0012.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
3056 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01000901.notes.0006.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
3057 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01000501.notes.0003.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
3058 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01001301.notes.0001.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
3059 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01000501.notes.0002.xml.jpg | 'utf-8' codec ca

ERROR:root:3060 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01000101.notes.0001.xml.jpg:
Traceback (most recent call last):
  File "/var/folders/zj/t8mz7g111911kcsdk6nwh8g00000gn/T/ipykernel_1963/64700856.py", line 5, in <module>
    html_content = obj['Body'].read().decode('utf-8')  # HTML as string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

ERROR:root:3061 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01002001.notes.0013.xml.jpg:
Traceback (most recent call last):
  File "/var/folders/zj/t8mz7g111911kcsdk6nwh8g00000gn/T/ipykernel_1963/64700856.py", line 5, in <module>
    html_content = obj['Body'].read().decode('utf-8')  # HTML as string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

Processing documents:   7%|▋         | 3062/46665 [12:49:15<47:31, 15.29it/s]ERROR:root:30

3060 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01000101.notes.0001.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
3061 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01002001.notes.0013.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
3062 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01002001.notes.0003.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
3063 | Error loading eu/LEG_EN_HTML_20250721_04_08/e6e6ee52-e3b7-4196-a846-6b8dffa5db52/xhtml/FOR-C_2007072EN.01000901.notes.0007.xml.jpg | 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


Processing documents:   7%|▋         | 3067/46665 [12:50:04<182:26:47, 15.07s/it]


KeyboardInterrupt: 

## Parallel

In [7]:
def parallel_extract(batch_func, df, collection, num_processes=4):
    chunks = np.array_split(df, num_processes)
    args = [(chunk, collection) for chunk in chunks]

    with Pool(processes=num_processes) as pool:
        pool.starmap(batch_func, args)

In [ ]:
from helper.chroma import process_documents_batch
parallel_extract(process_documents_batch, documents, collection, num_processes=4)

In [10]:
from concurrent.futures import ThreadPoolExecutor

def parallel_extract_threads(batch_func, df, collection, num_threads=4):
    chunks = np.array_split(df, num_threads)

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(batch_func, chunk, collection) for chunk in chunks]
        results = [f.result() for f in futures]

In [11]:
from helper.chroma import process_documents_batch
parallel_extract_threads(process_documents_batch, documents, collection, num_threads=4)

/Users/demouser/anaconda3/envs/regpy10ch/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/demouser/anaconda3/envs/regpy10ch/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/Users/demouser/anaconda3/envs/regpy10ch/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/demouser/anaconda3/envs/regpy10ch/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/Users/demouser/anaconda3/envs/regpy10ch/lib/python3.10/threading.py", line 953, in run
    self._target(

[99737] 3 | Error processing eu/LEG_EN_HTML_20250721_04_08/39bcdc85-2e3b-4e36-a488-5197ee502afd/html/31999Y0917_02_en.html | Numpy is not available
[99737] 2 | Error processing eu/LEG_EN_HTML_20250721_04_08/65a9ddff-3e78-403b-ab34-98d7973be2b3/xhtml/L_2010117EN.01006001.doc.html | Numpy is not available
[99737] 4 | Error processing eu/LEG_EN_HTML_20250721_04_08/a4d26b15-882d-11e9-9369-01aa75ed71a1/xhtml/L_2019148EN.01000101.doc.html | Numpy is not available
[99737] 0 | Error processing eu/LEG_EN_HTML_20250721_04_08/1a1e8486-a474-11e9-9d01-01aa75ed71a1/xhtml/L_2019187EN.01004101.doc.html | Numpy is not available
[99737] 1 | Error processing eu/LEG_EN_HTML_20250721_04_08/5fa72f58-9564-4ebe-a5a5-853e206ae2ed/html/32000R0212en.html | Numpy is not available


# Check chroma client

In [20]:
print("Number of embeddings:", collection.count())
results = collection.peek()
results['ids']

Number of embeddings: 118643


['9000786f2ac2_0',
 '9000786f2ac2_1',
 '9000786f2ac2_2',
 '9000786f2ac2_3',
 '9000786f2ac2_4',
 '9000786f2ac2_5',
 '9000786f2ac2_6',
 '9000786f2ac2_7',
 '9000786f2ac2_8',
 '9000786f2ac2_9']

In [21]:
all_ids = collection.get()["ids"]
doc_ids = set([i.split('_')[0] for i in all_ids])
len(doc_ids)

2674

In [25]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate

In [27]:
# Prompt template
prompt_template = """Answer the question in a concise manner based on the following context:
{context}

Question: {question}

If the context does not contain the answer, use other sources.
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
collection = ch_client.get_or_create_collection(name="legal")
if collection.count() == 0:
    print("Vectorstore is empty.")
else:
    print(f"Vectorstore loaded. [{len(doc_ids)} documents. {collection.count()} embeddings.]")
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
vectorstore = Chroma(
    client=ch_client,
    collection_name="legal",
    embedding_function=embeddings  # same embedding function used for inserts
)

In [38]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 5,
        "filter":{
            "celex-year": 2010
        }
    }
)
qa = RetrievalQA.from_chain_type(
    llm=llm,  # or whichever LLM
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

In [32]:
# Query it
response = qa.invoke("Which chemical was classified as a substance of very high concern for its endocrine-disrupting effects under the EU Commission Implementing Decision 2019/1194?")
print(response)

{'query': 'Which chemical was classified as a substance of very high concern for its endocrine-disrupting effects under the EU Commission Implementing Decision 2019/1194?', 'result': 'The chemical 4-tert-butylphenol (PTBP) was classified as a substance of very high concern for its endocrine-disrupting effects under the EU Commission Implementing Decision 2019/1194.', 'source_documents': [Document(metadata={'celex-year': 2019, 'celex-sector': '3', 'celex': '32019D1194', 'title': 'L_2019187EN.01004101.xml', 'eurovoc-terms': 'fish;endocrine disease;fish disease;chemical product;endocrine disruptor;environmental protection', 'eurovoc-mt': '5641 fisheries;2841 health;5631 agricultural activity;6811 chemistry;5216 deterioration of the environment;5206 environmental policy'}, page_content='HAS ADOPTED THIS DECISION:\n\n\n\n\n\n\n\n\nArticle 1\n\n\n\n\n1.\xa0\xa0\xa04-tert-butylphenol (PTBP) (EC No 202-679-0, CAS No 98-54-4) is identified as a substance of very high concern pursuant to Article

In [33]:
qn = "On what date did EU Regulation 402/2010 become effective?"
response = qa.invoke(qn)
response

{'query': 'On what date did EU Regulation 402/2010 become effective?',
 'result': 'The context provided does not mention EU Regulation 402/2010 or its effective date. Therefore, based on external sources, EU Regulation 402/2010 became effective on May 20, 2010.',
 'source_documents': [Document(metadata={'title': 'L_202402781EN.000101.fmx.xml', 'celex-year': 2024, 'celex-sector': '3', 'eurovoc-terms': 'market approval;plant health control;plant health product', 'eurovoc-mt': '2031 marketing;5606 agricultural policy;5626 means of agricultural production', 'celex': '32024R2781'}, page_content='HAS ADOPTED THIS REGULATION:\n\n\n\n\n\n\n\n\nArticle\xa01\n\n\nThe Annex to Implementing Regulation (EU) No\xa0540/2011 is amended in accordance with the Annex to this Regulation.\n\n\n\n\n\n\nArticle\xa02\n\n\nThis Regulation shall enter into force on the twentieth day following that of its publication in the \nOfficial Journal of the European Union\n.\n\n\n\n\n\n\n\n\n\n\nThis Regulation shall be

In [34]:
response['source_documents'][0].metadata['celex']

'32024R2781'

In [41]:
qn = "On what day should EU Regulation 402/2010 enter into force?"
response = qa.invoke(qn)
response

{'query': 'On what day should EU Regulation 402/2010 enter into force?',
 'result': 'EU Regulation 402/2010 should enter into force on the day following its publication in the Official Journal of the European Union.',
 'source_documents': [Document(metadata={'celex': '32010R0832', 'title': 'L_2010248EN.01000101.xml', 'celex-year': 2010, 'eurovoc-terms': 'management information system;Cohesion Fund;regional disparity;European Regional Development Fund;dissemination of information;administrative transparency;economic and social cohesion;European Social Fund;management audit', 'celex-sector': '3', 'eurovoc-mt': '4021 management;1021 EU finance;1616 regions and regional policy;3221 documentation;0436 executive power and public service;1016 European construction'}, page_content='(8)\n\n\n\n\n\n\nRegulation (EC) No 1828/2006 should therefore be amended accordingly.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n(9)\n\n\n\n\n\n\nFor reasons of coherence it is appropriate that the amendm

# To Solve
1. Long '\n' sequences in the document content
2. utf-8 error in processing

In [ ]:
structure = {'eu': ['eurlex-download', 'eurlex-sparql', 'eurlex-sample'], 'sg': ['mas', 'sso']}
for f in ['raw', 'processed']:
    for region, sources in structure.items():
        for source in sources:
            dir_path = f"data_ingestion/{f}/{region}/{source}"
            if not os.path.exists(dir_path):
                os.makedirs(dir_path)

In [43]:
for f in ['common', 'eu', 'sg', 'us', 'pipelines']:
    dir_path = f"data_ingestion/src/{f}"
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [4]:
for f in ['chroma']:
    for r in ['eu', 'sg', 'us']:
        dir_path = f"/Users/demouser/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Reg-Guru/backend/data_ingestion/{f}/{r}" 
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)